In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Задача 1.

Поиграемся с датасетом про диабет (он совсем легкий).

In [3]:
data = pd.read_csv('/content/gdrive/My Drive/data_stud/machlearn/Diabetes Binary Classification.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
X = data.drop('Class variable (0 or 1)', axis=1)
y = data['Class variable (0 or 1)']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [17]:
y.value_counts()

Class variable (0 or 1)
0    500
1    268
Name: count, dtype: int64

In [20]:
from sklearn.utils.class_weight import compute_class_weight

w = compute_class_weight(class_weight="balanced", classes=[0,1], y=y) #вычисление весов
print (w)
binclass=[0,1]
res = {binclass[i]: w[i] for i in range(len(binclass))} #делаем словарь...
print (res)

[0.768      1.43283582]
{0: 0.768, 1: 1.4328358208955223}


In [19]:
model = SVC(kernel='linear', class_weight=res)
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), '\n', classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       343
           1       0.76      0.65      0.70       233

    accuracy                           0.77       576
   macro avg       0.77      0.75      0.76       576
weighted avg       0.77      0.77      0.77       576
 
               precision    recall  f1-score   support

           0       0.80      0.82      0.81       122
           1       0.67      0.64      0.66        70

    accuracy                           0.76       192
   macro avg       0.74      0.73      0.73       192
weighted avg       0.75      0.76      0.75       192



Целевая переменная тут явно обозначена как Class variable (очевидно, есть у человека диабет или нет)

## Задача 2.

Второй датасет - про покупателей велосипедов.

In [25]:
bike = pd.read_csv('/content/gdrive/My Drive/data_stud/machlearn/bike_buyers_clean.csv')
bike.head()

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [28]:
print (bike['Marital Status'].unique())
print (bike['Education'].unique())
print (bike['Occupation'].unique())
print (bike['Home Owner'].unique())
print (bike['Commute Distance'].unique())
print (bike['Region'].unique())
print (bike['Purchased Bike'].unique())

['Married' 'Single']
['Bachelors' 'Partial College' 'High School' 'Partial High School'
 'Graduate Degree']
['Skilled Manual' 'Clerical' 'Professional' 'Manual' 'Management']
['Yes' 'No']
['0-1 Miles' '2-5 Miles' '5-10 Miles' '1-2 Miles' '10+ Miles']
['Europe' 'Pacific' 'North America']
['No' 'Yes']


In [29]:
bike = bike.drop (['ID'], axis=1)

#marital status
bike.loc[bike["Marital Status"] == "Married", "Marital Status"] = 0
bike.loc[bike["Marital Status"] == "Single", "Marital Status"] = 1
#gender
bike.loc[bike["Gender"] == "Male", "Gender"] = 0
bike.loc[bike["Gender"] == "Female", "Gender"] = 1
#education
bike.loc[bike["Education"] == "Bachelors", "Education"] = 0
bike.loc[bike["Education"] == "Partial College", "Education"] = 1
bike.loc[bike["Education"] == "High School", "Education"] = 2
bike.loc[bike["Education"] == "Partial High School", "Education"] = 3
bike.loc[bike["Education"] == "Graduate Degree", "Education"] = 4
#occupation
bike.loc[bike["Occupation"] == "Skilled Manual", "Occupation"] = 0
bike.loc[bike["Occupation"] == "Clerical", "Occupation"] = 1
bike.loc[bike["Occupation"] == "Professional", "Occupation"] = 2
bike.loc[bike["Occupation"] == "Manual", "Occupation"] = 3
bike.loc[bike["Occupation"] == "Management", "Occupation"] = 4
#home owner
bike.loc[bike["Home Owner"] == "No", "Home Owner"] = 0
bike.loc[bike["Home Owner"] == "Yes", "Home Owner"] = 1
#commute distance
bike.loc[bike["Commute Distance"] == "0-1 Miles", "Commute Distance"] = 0
bike.loc[bike["Commute Distance"] == "2-5 Miles", "Commute Distance"] = 1
bike.loc[bike["Commute Distance"] == "5-10 Miles", "Commute Distance"] = 2
bike.loc[bike["Commute Distance"] == "1-2 Miles", "Commute Distance"] = 3
bike.loc[bike["Commute Distance"] == "10+ Miles", "Commute Distance"] = 4
#region
bike.loc[bike["Region"] == "Europe", "Region"] = 0
bike.loc[bike["Region"] == "Pacific", "Region"] = 1
bike.loc[bike["Region"] == "North America", "Region"] = 2
#purchased bike
bike.loc[bike["Purchased Bike"] == "No", "Purchased Bike"] = 0
bike.loc[bike["Purchased Bike"] == "Yes", "Purchased Bike"] = 1

bike.head ()

,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,0,1,40000,1,0,0,1,0,0,0,42,0
1,0,0,30000,3,1,1,1,1,0,0,43,0
2,0,0,80000,5,1,2,0,2,1,0,60,0
3,1,0,70000,0,0,2,1,1,2,1,41,1
4,1,0,30000,0,0,1,0,0,0,0,36,1


In [32]:
salary = bike['Income'].tolist()
i = 0
while (i < len(salary)):
  salary[i] = salary[i]/10000 #равномерно уменьшил доход
  i+=1

salary = list(map(int, salary))
bike['Income'] = salary
bike.head ()

,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,0,1,4,1,0,0,1,0,0,0,42,0
1,0,0,3,3,1,1,1,1,0,0,43,0
2,0,0,8,5,1,2,0,2,1,0,60,0
3,1,0,7,0,0,2,1,1,2,1,41,1
4,1,0,3,0,0,1,0,0,0,0,36,1


In [63]:
X = bike.drop('Purchased Bike', axis=1)
y = bike['Purchased Bike'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y.value_counts()

Purchased Bike
0    519
1    481
Name: count, dtype: int64

In [64]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), '\n', classification_report(ypred_test, y_test))

#пробовал менять понемногу всё, но результаты всегда примерно такими оставались. Слишком маленький датасет?

              precision    recall  f1-score   support

           0       0.65      0.64      0.65       417
           1       0.62      0.63      0.62       383

    accuracy                           0.64       800
   macro avg       0.64      0.64      0.64       800
weighted avg       0.64      0.64      0.64       800
 
               precision    recall  f1-score   support

           0       0.68      0.70      0.69       107
           1       0.64      0.61      0.63        93

    accuracy                           0.66       200
   macro avg       0.66      0.66      0.66       200
weighted avg       0.66      0.66      0.66       200



Пытаемся по характеристикам человека понять, купит он велик или нет.